# 📊 Stock Performance & Investment Analysis (2025)
### Portfolio Optimization · Risk-Adjusted Returns · Investment Strategy Comparison · Financial Modeling

---

I wanted to build a project that combined financial modeling with the SQL and Python skills I've been developing in class. The idea is to analyze stock performance across a few different sectors, then actually try to build and optimize portfolios — not just look at returns, but really dig into the risk-adjusted side of things.

**Technologies:** Python (pandas, NumPy, SciPy, Matplotlib, Seaborn) · SQL (SQLite)

**Key Techniques:** Markowitz Portfolio Optimization · Sharpe & Sortino Ratios · Value at Risk (VaR) · Maximum Drawdown · Momentum & Risk-Parity Strategies · Cholesky Decomposition for Correlated Return Simulation

---

### Table of Contents
1. [Setup & Configuration](#1-setup--configuration)
2. [Data Acquisition & Validation](#2-data-acquisition--validation)
3. [SQL Database Layer](#3-sql-database-layer--storage--querying)
4. [Exploratory Performance Analysis](#4-exploratory-performance-analysis)
5. [Correlation & Sector Analysis](#5-correlation--sector-analysis)
6. [Portfolio Optimization (Markowitz)](#6-portfolio-optimization-markowitz-mean-variance)
7. [Risk Metrics & Drawdown Analysis](#7-risk-metrics--drawdown-analysis)
8. [Investment Strategy Comparison](#8-investment-strategy-comparison)
9. [Executive Summary](#9-executive-summary)


---
## 1. Setup & Configuration

I'm starting by pulling in my libraries and laying out the stock universe I want to work with. I picked 8 stocks spread across 4 sectors — tech, financials, healthcare, energy, and consumer staples — plus SPY as my market benchmark. That way I have enough diversity to actually see differences when I start comparing portfolios later.


In [ ]:
# ============================================================
# SETUP & CONFIGURATION
# ============================================================
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.optimize import minimize
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# Setting the seed so my results are reproducible every time I re-run
np.random.seed(42)

# ─── Style Configuration ───
# I'm going to use a consistent color palette across all my plots
plt.style.use('seaborn-v0_8-whitegrid')
COLORS = {
    'primary':   '#1a365d',   # deep navy
    'accent':    '#e53e3e',   # alert red
    'positive':  '#276749',   # green
    'negative':  '#c53030',   # red
    'muted':     '#718096',   # gray
    'palette':   ['#1a365d', '#2b6cb0', '#3182ce', '#63b3ed',
                  '#e53e3e', '#276749', '#d69e2e', '#805ad5']
}

# ─── Stock Universe ───
# I picked 8 stocks across different sectors to get real diversification.
# For each one I'm defining mu (expected annual return), sigma (annual volatility),
# and a starting price. These params drive the simulation in the next cell.
STOCK_UNIVERSE = {
    'AAPL':  {'mu': 0.24, 'sigma': 0.28, 'sector': 'Technology',        'price_start': 195},
    'MSFT':  {'mu': 0.21, 'sigma': 0.24, 'sector': 'Technology',        'price_start': 420},
    'GOOGL': {'mu': 0.18, 'sigma': 0.26, 'sector': 'Technology',        'price_start': 190},
    'JPM':   {'mu': 0.15, 'sigma': 0.22, 'sector': 'Financials',        'price_start': 230},
    'V':     {'mu': 0.17, 'sigma': 0.19, 'sector': 'Financials',        'price_start': 290},
    'JNJ':   {'mu': 0.08, 'sigma': 0.17, 'sector': 'Healthcare',        'price_start': 155},
    'XOM':   {'mu': 0.12, 'sigma': 0.25, 'sector': 'Energy',            'price_start': 105},
    'PG':    {'mu': 0.10, 'sigma': 0.15, 'sector': 'Consumer Staples',  'price_start': 162},
}

# SPY benchmark params — I'm keeping its volatility lower than most individual stocks,
# which makes sense since it's a diversified index
BENCHMARK_PARAMS = {'mu': 0.14, 'sigma': 0.15, 'price_start': 590}
TICKERS = list(STOCK_UNIVERSE.keys())
RISK_FREE_RATE = 0.045  # using the approximate 2025 risk-free rate

print("✓ Configuration loaded")
print(f"  Stock universe: {len(TICKERS)} equities across "
      f"{len(set(s['sector'] for s in STOCK_UNIVERSE.values()))} sectors + SPY benchmark")
print(f"  Risk-free rate: {RISK_FREE_RATE*100}%")


---
## 2. Data Acquisition & Validation

In a real production environment I'd be pulling this from yfinance or a broker API, but for the portfolio I'm simulating the price data using **Cholesky decomposition**. I chose this approach because it lets me bake in realistic correlations between stocks — like the fact that AAPL and MSFT should move together way more than AAPL and PG. That matters a lot once I get to the optimization step.

After I generate the data, I'm running a quick validation pass to make sure everything looks reasonable before I start analyzing it.


In [ ]:
# ============================================================
# DATA ACQUISITION — Simulated with Cholesky Decomposition
# ============================================================
# In production I'd swap this out for yfinance:
#   import yfinance as yf
#   df_prices = yf.download(TICKERS + ['SPY'], start='2025-01-01', end='2025-12-31')['Close']

# ─── Step 1: Define the correlation structure ───
# I'm hard-coding a correlation matrix that reflects how these stocks actually
# relate to each other. Tech names cluster together, financials pair up,
# and the defensive names like JNJ and PG are pretty isolated.
CORR_MATRIX = np.array([
#        AAPL  MSFT  GOOGL  JPM   V     JNJ   XOM   PG
        [1.00, 0.76, 0.73,  0.44, 0.39, 0.14, 0.19, 0.09],  # AAPL
        [0.76, 1.00, 0.71,  0.41, 0.37, 0.17, 0.17, 0.11],  # MSFT
        [0.73, 0.71, 1.00,  0.39, 0.34, 0.13, 0.15, 0.07],  # GOOGL
        [0.44, 0.41, 0.39,  1.00, 0.64, 0.24, 0.29, 0.14],  # JPM
        [0.39, 0.37, 0.34,  0.64, 1.00, 0.19, 0.24, 0.17],  # V
        [0.14, 0.17, 0.13,  0.24, 0.19, 1.00, 0.09, 0.34],  # JNJ
        [0.19, 0.17, 0.15,  0.29, 0.24, 0.09, 1.00, 0.04],  # XOM
        [0.09, 0.11, 0.07,  0.14, 0.17, 0.34, 0.04, 1.00],  # PG
])

# ─── Step 2: Cholesky decomposition to generate correlated noise ───
# This is the key trick — I decompose the correlation matrix into a lower-triangular
# matrix L, then I can multiply uncorrelated random draws by L to get correlated ones.
# I learned about this in class and it's a really clean way to simulate multi-asset returns.
L = np.linalg.cholesky(CORR_MATRIX)

trading_days = pd.bdate_range(start='2025-01-02', end='2025-12-31')
n_days = len(trading_days)
n_stocks = len(TICKERS)

# Converting annual params to daily by dividing mu by 252 and sigma by sqrt(252)
daily_mu    = np.array([STOCK_UNIVERSE[t]['mu']    for t in TICKERS]) / 252
daily_sigma = np.array([STOCK_UNIVERSE[t]['sigma'] for t in TICKERS]) / np.sqrt(252)

# I generate uncorrelated standard-normal shocks first, then apply L to correlate them
Z_uncorr   = np.random.randn(n_days, n_stocks)
Z_corr     = Z_uncorr @ L.T                          # now they're correlated
daily_rets = daily_mu + daily_sigma * Z_corr          # daily return = drift + vol * noise

# ─── Step 3: Build price series from the daily returns ───
# I'm using cumulative product here — each day's price is last day's price * (1 + return)
prices = {}
for i, ticker in enumerate(TICKERS):
    prices[ticker] = STOCK_UNIVERSE[ticker]['price_start'] * np.cumprod(1 + daily_rets[:, i])

# ─── Step 4: Generate SPY benchmark ───
# I'm tying SPY's movement to the average of all stocks (correlation ~0.7)
# with some independent noise mixed in, which felt like a realistic way to simulate it
market_avg_ret = daily_rets.mean(axis=1)
spy_noise      = np.random.randn(n_days)
spy_daily_ret  = (0.70 * market_avg_ret +
                  0.30 * (BENCHMARK_PARAMS['mu']/252 + BENCHMARK_PARAMS['sigma']/np.sqrt(252) * spy_noise))
prices['SPY'] = BENCHMARK_PARAMS['price_start'] * np.cumprod(1 + spy_daily_ret)

# ─── Assemble everything into one clean DataFrame ───
df_prices = pd.DataFrame(prices, index=trading_days)
df_prices.index.name = 'Date'

print("✓ Price data generated")
print(f"  Trading days: {n_days}  |  Stocks: {n_stocks + 1} (incl. SPY)")
df_prices.head()


In [ ]:
# ============================================================
# DATA VALIDATION — Making sure the simulated data looks right
# ============================================================
# I always want to sanity-check my data before I start analyzing it.
# I'm going to check for missing values, compare realized returns to my target mus,
# and verify the correlation matrix is well-formed.
df_returns = df_prices.pct_change().dropna()  # daily returns

print("=" * 60)
print("  DATA VALIDATION REPORT")
print("=" * 60)

# Check 1: No missing values — should be zero since I generated the data myself
missing = df_prices.isnull().sum().sum()
print(f"\n  Missing values:        {missing} {'✓' if missing == 0 else '✗ ALERT'}")

# Check 2: Comparing what I got vs what I expected for each stock's annual return.
# With only 252 trading days of noise, I don't expect these to be perfect matches,
# so I'm flagging anything that's more than 15% off from the target mu.
realized_annual_ret = ((df_prices.iloc[-1] / df_prices.iloc[0]) - 1)
print(f"\n  {'Ticker':<8} {'Target μ':>10} {'Realized':>10} {'Within 15%?':>12}")
print(f"  {'─'*8} {'─'*10} {'─'*10} {'─'*12}")
for t in TICKERS:
    target = STOCK_UNIVERSE[t]['mu']
    actual = realized_annual_ret[t]
    flag   = '✓' if abs(actual - target) < 0.15 else '⚠'
    print(f"  {t:<8} {target:>9.1%} {actual:>9.1%} {flag:>12}")

# Check 3: The correlation matrix should be symmetric and all values between -1 and 1
realized_corr = df_returns[TICKERS].corr()
print(f"\n  Correlation matrix symmetric: {'✓' if np.allclose(realized_corr, realized_corr.T) else '✗'}")
print(f"  All correlations in [-1, 1]:  {'✓' if realized_corr.values.min() >= -1 and realized_corr.values.max() <= 1 else '✗'}")
print("=" * 60)


---
## 3. SQL Database Layer — Storage & Querying

I wanted to make sure this project actually uses SQL in a meaningful way, not just as a pass-through. So I'm setting up a normalized SQLite database — three tables: prices, metadata, and precomputed daily returns. Then I'm writing queries to pull out the business-level aggregations I need: quarterly breakdowns, sector-level performance, and who the top performer was each quarter.

This is the kind of data management workflow I've been practicing in OPAN 6608, and I think it makes the project feel more like something you'd actually do at a company.


In [ ]:
# ============================================================
# SQL LAYER — Creating the database and loading the data in
# ============================================================
# I'm setting up an in-memory SQLite database with a normalized schema.
# Three tables: prices, metadata, and precomputed returns.
# In production I'd write this to a file or connect to a real data warehouse.
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# ─── Create the tables ───
cursor.executescript("""
    -- Daily closing prices for every stock
    CREATE TABLE stock_prices (
        price_date  TEXT NOT NULL,
        ticker      TEXT NOT NULL,
        close_price REAL NOT NULL,
        PRIMARY KEY (price_date, ticker)
    );

    -- Maps each ticker to its sector — keeping this separate is good practice
    CREATE TABLE stock_metadata (
        ticker  TEXT PRIMARY KEY,
        sector  TEXT NOT NULL
    );

    -- I'm precomputing daily returns so my SQL queries don't have to
    CREATE TABLE daily_returns (
        return_date TEXT NOT NULL,
        ticker      TEXT NOT NULL,
        daily_return REAL NOT NULL,
        PRIMARY KEY (return_date, ticker)
    );
""")

# ─── Load everything in ───
# Prices — one row per stock per day
prices_records = [
    (str(date.date()), ticker, round(price, 2))
    for date in df_prices.index
    for ticker, price in df_prices.loc[date].items()
]
cursor.executemany("INSERT INTO stock_prices VALUES (?, ?, ?)", prices_records)

# Metadata — just the 8 stocks, not SPY
meta_records = [(t, STOCK_UNIVERSE[t]['sector']) for t in TICKERS]
cursor.executemany("INSERT INTO stock_metadata VALUES (?, ?)", meta_records)

# Returns — excluding SPY here since I track the benchmark separately
rets_records = [
    (str(date.date()), ticker, round(ret, 6))
    for date in df_returns.index
    for ticker, ret in df_returns.loc[date].items()
    if ticker != 'SPY'
]
cursor.executemany("INSERT INTO daily_returns VALUES (?, ?, ?)", rets_records)

conn.commit()
print(f"✓ Database populated")
print(f"  stock_prices:  {cursor.execute('SELECT COUNT(*) FROM stock_prices').fetchone()[0]:,} rows")
print(f"  daily_returns: {cursor.execute('SELECT COUNT(*) FROM daily_returns').fetchone()[0]:,} rows")


In [ ]:
# ============================================================
# SQL QUERIES — Now let me pull out some useful aggregations
# ============================================================

# ── Query 1: Full-year return and price range per stock ──
# I want to see who had the best annualized return at a glance.
# I'm also grabbing the price range (max - min) to get a feel for volatility.
query_1 = """
    SELECT
        ticker,
        ROUND(AVG(daily_return) * 252, 4)                          AS annualized_avg_return,
        ROUND(
            (MAX(close_price) - MIN(close_price)) / MIN(close_price),
            4
        )                                                          AS price_range_pct
    FROM daily_returns
    JOIN stock_prices USING (ticker)
    GROUP BY ticker
    ORDER BY annualized_avg_return DESC
"""
print("── Query 1: Annualized Returns & Price Range ──\n")
df_q1 = pd.read_sql(query_1, conn)
df_q1.columns = ['Ticker', 'Annualized Return', 'Price Range %']
print(df_q1.to_string(index=False))

# ── Query 2: Sector-level performance ──
# I'm curious which sector did best on average. I'm joining to the metadata table
# to get the sector labels, then grouping and averaging.
query_2 = """
    SELECT
        m.sector,
        COUNT(DISTINCT r.ticker)                                   AS num_stocks,
        ROUND(AVG(r.daily_return) * 252, 4)                        AS sector_avg_annual_return,
        ROUND(GROUP_CONCAT(DISTINCT r.ticker), 100)                AS tickers
    FROM daily_returns r
    JOIN stock_metadata m ON r.ticker = m.ticker
    GROUP BY m.sector
    ORDER BY sector_avg_annual_return DESC
"""
print("\n── Query 2: Sector Performance Summary ──\n")
df_q2 = pd.read_sql(query_2, conn)
print(df_q2.to_string(index=False))


In [ ]:
# ── Query 3: Quarterly performance breakdown ──
# I want to see how each stock did quarter by quarter — this is where
# I'm using CASE + SUBSTR to assign quarters from the date string.
# That's a pattern I picked up from class that's surprisingly handy.
query_3 = """
    WITH quarterly AS (
        SELECT
            ticker,
            CASE
                WHEN SUBSTR(return_date, 6, 2) IN ('01','02','03') THEN 'Q1'
                WHEN SUBSTR(return_date, 6, 2) IN ('04','05','06') THEN 'Q2'
                WHEN SUBSTR(return_date, 6, 2) IN ('07','08','09') THEN 'Q3'
                ELSE                                                     'Q4'
            END AS quarter,
            daily_return
        FROM daily_returns
    )
    SELECT
        ticker,
        quarter,
        ROUND(SUM(daily_return), 4)  AS quarterly_return
    FROM quarterly
    GROUP BY ticker, quarter
    ORDER BY ticker, quarter
"""
print("── Query 3: Quarterly Return Breakdown ──\n")
df_q3 = pd.read_sql(query_3, conn)
# Pivoting this into a nice table so I can read it easily
df_q3_pivot = df_q3.pivot(index='ticker', columns='quarter', values='quarterly_return')
df_q3_pivot = df_q3_pivot[['Q1', 'Q2', 'Q3', 'Q4']]  # enforce the right order
df_q3_pivot['Full Year'] = df_q3_pivot.sum(axis=1).round(4)
print(df_q3_pivot.to_string())

# ── Query 4: Top performer each quarter ──
# This one uses a correlated subquery to find the max return per quarter.
# I wanted to see if the same stock dominated all year or if it shifted around.
query_4 = """
    WITH quarterly AS (
        SELECT
            ticker,
            CASE
                WHEN SUBSTR(return_date, 6, 2) IN ('01','02','03') THEN 'Q1'
                WHEN SUBSTR(return_date, 6, 2) IN ('04','05','06') THEN 'Q2'
                WHEN SUBSTR(return_date, 6, 2) IN ('07','08','09') THEN 'Q3'
                ELSE                                                     'Q4'
            END AS quarter,
            daily_return
        FROM daily_returns
    ),
    q_totals AS (
        SELECT ticker, quarter, ROUND(SUM(daily_return), 4) AS qret
        FROM quarterly
        GROUP BY ticker, quarter
    )
    SELECT quarter, ticker AS top_performer, qret AS return
    FROM q_totals
    WHERE qret = (SELECT MAX(qret) FROM q_totals AS t2 WHERE t2.quarter = q_totals.quarter)
    ORDER BY quarter
"""
print("\n── Query 4: Top Performer by Quarter ──\n")
df_q4 = pd.read_sql(query_4, conn)
print(df_q4.to_string(index=False))


---
## 4. Exploratory Performance Analysis

Before I jump into any heavy modeling, I want to just look at how these stocks actually performed over the year. I'm normalizing everything to $100 at the start so I can compare them on the same scale, and plotting them against SPY so I can immediately see who beat the market and by how much.


In [ ]:
# ============================================================
# EXPLORATORY ANALYSIS — Let me visualize cumulative returns
# ============================================================
# I'm normalizing all prices to $100 at the start of the year.
# That way I can plot them all on the same scale and see
# who actually did the most relative to where they started.
df_normalized = (df_prices / df_prices.iloc[0]) * 100

fig, ax = plt.subplots(figsize=(14, 7))

# SPY goes in first as the benchmark reference line — thick and dashed
ax.plot(df_normalized.index, df_normalized['SPY'],
        color=COLORS['muted'], linewidth=2.5, linestyle='--', label='SPY (Benchmark)', zorder=5)

# Plotting each stock with its own color
for i, ticker in enumerate(TICKERS):
    ax.plot(df_normalized.index, df_normalized[ticker],
            color=COLORS['palette'][i], linewidth=1.6, alpha=0.85, label=ticker)

# I'm adding the final value as a label on the right side of each line
# so it's easy to read the ending value without hovering
final_vals = df_normalized.iloc[-1].sort_values(ascending=False)
for rank, (ticker, val) in enumerate(final_vals.items()):
    ax.annotate(f'{val:.0f}',
                xy=(df_normalized.index[-1], val),
                xytext=(8, 0), textcoords='offset points',
                fontsize=8, fontweight='bold',
                color=COLORS['primary'] if ticker != 'SPY' else COLORS['muted'],
                va='center')

ax.set_title('2025 Cumulative Returns — Normalized to $100', fontsize=16, fontweight='bold', pad=15)
ax.set_ylabel('Value ($)', fontsize=12)
ax.set_xlabel('')
ax.legend(loc='upper left', fontsize=9, framealpha=0.95)
ax.set_xlim(df_normalized.index[0], df_normalized.index[-1])
ax.axhline(100, color='gray', linewidth=0.8, linestyle=':')  # baseline

plt.tight_layout()
plt.savefig('01_cumulative_returns.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 01_cumulative_returns.png")


In [ ]:
# ─── Performance Summary Table ───
# Let me put together a clean summary table with all the key metrics side by side.
# I want return, volatility, and average daily return so I can see the tradeoffs.
full_year_return   = (df_prices.iloc[-1] / df_prices.iloc[0] - 1) * 100
annualized_vol     = df_returns.std() * np.sqrt(252) * 100
avg_daily_return   = df_returns.mean() * 100

summary = pd.DataFrame({
    'Sector':        [STOCK_UNIVERSE.get(t, {}).get('sector', 'Index') for t in df_prices.columns],
    'Start Price':   df_prices.iloc[0].round(2),
    'End Price':     df_prices.iloc[-1].round(2),
    'Return (%)':    full_year_return.round(2),
    'Volatility (%)': annualized_vol.round(2),
    'Avg Daily Ret (%)': avg_daily_return.round(3)
}).sort_values('Return (%)', ascending=False)

print("── 2025 Performance Summary ──\n")
print(summary.to_string())


---
## 5. Correlation & Sector Analysis

I'm curious how much these stocks actually move together — that's going to be a big deal for the portfolio optimization coming up next. I'm going to visualize the correlation matrix as a heatmap, and then I also want to see if I can break down the returns by sector. My hypothesis is that the tech stocks are going to be pretty tightly correlated with each other, and the defensive names like JNJ and PG will be more isolated. Let me see if that holds up.


In [ ]:
# ============================================================
# CORRELATION HEATMAP & SECTOR PERFORMANCE
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6.5),
                         gridspec_kw={'width_ratios': [1.1, 0.9]})

# ─── Left panel: correlation heatmap ───
# I'm only including the 8 stocks here, not SPY — I want to see
# how the individual names relate to each other before I optimize.
corr = df_returns[TICKERS].corr()
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)  # upper triangle for masking

sns.heatmap(corr, ax=axes[0], annot=True, fmt='.2f', cmap='RdYlBu_r',
            vmin=-0.2, vmax=1.0, linewidths=0.5, linecolor='white',
            cbar_kws={'shrink': 0.85, 'label': 'Pearson Correlation'},
            annot_kws={'size': 9})
axes[0].set_title('Cross-Stock Correlation Matrix', fontsize=14, fontweight='bold')
axes[0].tick_params(labelsize=10)

# ─── Right panel: average return by sector ───
# Grouping by sector and averaging — I want to see which sector
# was the real driver of returns this year.
sector_map   = {t: STOCK_UNIVERSE[t]['sector'] for t in TICKERS}
sector_rets  = full_year_return[TICKERS].groupby(sector_map).mean().sort_values(ascending=True)

bars = axes[1].barh(sector_rets.index, sector_rets.values,
                    color=[COLORS['positive'] if v > 0 else COLORS['negative'] for v in sector_rets.values],
                    edgecolor='white', linewidth=1.2, height=0.5)

# Adding value labels so I don't have to eyeball the axis
for bar, val in zip(bars, sector_rets.values):
    axes[1].text(val + 0.3, bar.get_y() + bar.get_height()/2,
                 f'{val:.1f}%', va='center', fontsize=10, fontweight='bold')

axes[1].axvline(0, color='gray', linewidth=0.8)
axes[1].set_title('Average Return by Sector (2025)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Average Return (%)', fontsize=11)
axes[1].set_xlim(sector_rets.min() - 3, sector_rets.max() + 4)
axes[1].tick_params(labelsize=10)

plt.tight_layout()
plt.savefig('02_correlation_sector.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 02_correlation_sector.png")


---
## 6. Portfolio Optimization — Markowitz Mean-Variance

This is the part I've been most excited to build. I'm going to use **Markowitz Modern Portfolio Theory** to actually solve for two key portfolios:
1. The **Minimum Variance Portfolio** — the one that gives me the lowest possible risk
2. The **Maximum Sharpe Ratio Portfolio** — the one that gives me the best return per unit of risk

I also want to trace out the full **Efficient Frontier** — the curve of all the optimal portfolios in between. I'm constraining it to long-only (no short selling), with each stock between 1% and 40% weight, so the results feel realistic.


In [ ]:
# ============================================================
# PORTFOLIO OPTIMIZATION — Markowitz Mean-Variance
# ============================================================
# First I need the annualized return vector and covariance matrix from my 2025 data.
# Everything below builds on these two inputs.
stock_returns = df_returns[TICKERS]
annual_returns = stock_returns.mean() * 252           # expected return vector
cov_matrix     = stock_returns.cov() * 252            # annualized covariance matrix
n              = len(TICKERS)

# ─── Helper functions for portfolio math ───
# I'm wrapping these up so I can reuse them in the optimizer cleanly.
def portfolio_return(weights):
    """Expected annual return given a weight vector."""
    return weights @ annual_returns.values

def portfolio_volatility(weights):
    """Annual standard deviation of the portfolio."""
    return np.sqrt(weights @ cov_matrix.values @ weights)

def neg_sharpe(weights):
    """I negate the Sharpe ratio because scipy only minimizes."""
    return -(portfolio_return(weights) - RISK_FREE_RATE) / portfolio_volatility(weights)

# ─── Setting up constraints and bounds ───
# Weights have to sum to 1, and I'm capping each stock between 1% and 40%.
# The 40% cap prevents the optimizer from just putting everything into one stock.
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds       = tuple((0.01, 0.40) for _ in range(n))
initial_w    = np.array([1/n] * n)  # starting guess: equal weight

# ─── Solve 1: Minimum Variance Portfolio ───
# I'm asking scipy to minimize volatility — this should push allocation
# toward the less correlated, lower-vol names like PG and JNJ.
min_var_result = minimize(portfolio_volatility, initial_w,
                          method='SLSQP', bounds=bounds, constraints=constraints)
min_var_weights = min_var_result.x

# ─── Solve 2: Maximum Sharpe Portfolio ───
# This one maximizes return per unit of risk. I expect it to tilt
# toward the higher-returning stocks while still diversifying.
max_sharpe_result = minimize(neg_sharpe, initial_w,
                             method='SLSQP', bounds=bounds, constraints=constraints)
max_sharpe_weights = max_sharpe_result.x

print("── Optimization Results ──\n")
print(f"  {'Metric':<25} {'Min Variance':>14} {'Max Sharpe':>14}")
print(f"  {'─'*25} {'─'*14} {'─'*14}")
print(f"  {'Expected Return':<25} {portfolio_return(min_var_weights):>13.2%} {portfolio_return(max_sharpe_weights):>13.2%}")
print(f"  {'Volatility':<25} {portfolio_volatility(min_var_weights):>13.2%} {portfolio_volatility(max_sharpe_weights):>13.2%}")
sharpe_mv = (portfolio_return(min_var_weights) - RISK_FREE_RATE) / portfolio_volatility(min_var_weights)
sharpe_ms = (portfolio_return(max_sharpe_weights) - RISK_FREE_RATE) / portfolio_volatility(max_sharpe_weights)
print(f"  {'Sharpe Ratio':<25} {sharpe_mv:>14.3f} {sharpe_ms:>14.3f}")


In [ ]:
# ─── Generating the Efficient Frontier ───
# Now I want to trace out the full curve of optimal portfolios.
# I'm sweeping across a range of target returns and for each one,
# I ask the optimizer: what's the minimum volatility I can achieve?
# The collection of all those points forms the efficient frontier.
target_returns = np.linspace(
    portfolio_return(min_var_weights),
    annual_returns.max() * 0.85,  # I'm capping this below the single best stock
    60                             # 60 points gives a smooth curve
)

frontier_vols, frontier_rets, frontier_weights = [], [], []

for target in target_returns:
    # Two constraints now: weights sum to 1, AND return hits my target
    constraints_ef = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
        {'type': 'eq', 'fun': lambda w, t=target: portfolio_return(w) - t}
    ]
    res = minimize(portfolio_volatility, initial_w,
                   method='SLSQP', bounds=bounds, constraints=constraints_ef)
    if res.success:
        frontier_vols.append(portfolio_volatility(res.x))
        frontier_rets.append(portfolio_return(res.x))
        frontier_weights.append(res.x)

frontier_vols = np.array(frontier_vols)
frontier_rets = np.array(frontier_rets)

# I also want to scatter the individual stocks on the same plot
# so I can see where each one falls relative to the frontier
stock_vols = np.array([cov_matrix.loc[t, t]**0.5 for t in TICKERS])
stock_rets = annual_returns.values

print(f"✓ Efficient frontier computed: {len(frontier_vols)} portfolios")


In [ ]:
# ─── Plotting the Efficient Frontier and Optimal Portfolios ───
fig, axes = plt.subplots(1, 2, figsize=(16, 6.5))

# ── Left panel: the frontier itself ──
ax = axes[0]
ax.plot(frontier_vols * 100, frontier_rets * 100,
        color=COLORS['primary'], linewidth=2.5, label='Efficient Frontier')

# Scatter each individual stock so I can see where they land
ax.scatter(stock_vols * 100, stock_rets * 100,
           c=COLORS['palette'][:n], s=100, edgecolors='white', linewidths=1.5, zorder=5)
for i, t in enumerate(TICKERS):
    ax.annotate(t, (stock_vols[i]*100, stock_rets[i]*100),
                xytext=(6, 4), textcoords='offset points', fontsize=9, fontweight='bold')

# Marking the two key portfolios I solved for
ax.scatter(portfolio_volatility(min_var_weights)*100, portfolio_return(min_var_weights)*100,
           marker='*', s=300, color=COLORS['accent'], edgecolors='white', linewidths=1.5,
           zorder=6, label='Min Variance')

ax.scatter(portfolio_volatility(max_sharpe_weights)*100, portfolio_return(max_sharpe_weights)*100,
           marker='D', s=150, color=COLORS['positive'], edgecolors='white', linewidths=1.5,
           zorder=6, label='Max Sharpe')

# Drawing the Capital Market Line — this is the line from the risk-free rate
# through the Max Sharpe portfolio. Everything below it is suboptimal.
cml_x = np.linspace(0, max(frontier_vols)*100 * 1.1, 50)
cml_slope = (portfolio_return(max_sharpe_weights) - RISK_FREE_RATE) / portfolio_volatility(max_sharpe_weights)
cml_y = RISK_FREE_RATE * 100 + cml_slope * cml_x
ax.plot(cml_x, cml_y, color=COLORS['muted'], linewidth=1.2, linestyle=':', alpha=0.7, label='Capital Market Line')

ax.set_xlabel('Annualized Volatility (%)', fontsize=11)
ax.set_ylabel('Annualized Return (%)', fontsize=11)
ax.set_title('Efficient Frontier with Optimal Portfolios', fontsize=14, fontweight='bold')
ax.legend(fontsize=9, framealpha=0.95)
ax.set_xlim(0, max(frontier_vols)*100 * 1.15)

# ── Right panel: comparing the actual weight allocations ──
# I want to see side by side how Min Variance vs Max Sharpe allocate differently.
# I'm also drawing a dashed line at equal weight for reference.
ax2 = axes[1]
x_pos = np.arange(n)
width = 0.35
bars1 = ax2.bar(x_pos - width/2, min_var_weights * 100,  width, label='Min Variance',  color=COLORS['accent'],   edgecolor='white')
bars2 = ax2.bar(x_pos + width/2, max_sharpe_weights * 100, width, label='Max Sharpe',   color=COLORS['positive'], edgecolor='white')

ax2.set_xticks(x_pos)
ax2.set_xticklabels(TICKERS, fontsize=10)
ax2.set_ylabel('Weight (%)', fontsize=11)
ax2.set_title('Optimal Portfolio Allocations', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.axhline(100/n, color=COLORS['muted'], linewidth=1, linestyle='--', alpha=0.6)
ax2.text(n-0.5, 100/n + 0.5, 'Equal Weight', fontsize=8, color=COLORS['muted'], ha='right')

# Value labels on each bar
for bar in bars1:
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=7.5, fontweight='bold')
for bar in bars2:
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{bar.get_height():.1f}%', ha='center', va='bottom', fontsize=7.5, fontweight='bold')

plt.tight_layout()
plt.savefig('03_efficient_frontier.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 03_efficient_frontier.png")


---
## 7. Risk Metrics & Drawdown Analysis

Return numbers alone don't tell the whole story — I need to understand the risk side too. I'm going to compute a full set of risk-adjusted metrics for every stock and for SPY:

| Metric | What it tells me |
|---|---|
| **Sharpe Ratio** | How much excess return I'm getting per unit of total risk |
| **Sortino Ratio** | Same idea, but it only penalizes me for *downside* risk — I like this one better |
| **Value at Risk (VaR 95%)** | On a bad day, how much could I lose? (at 95% confidence) |
| **Max Drawdown** | The worst peak-to-trough drop over the whole year |
| **Calmar Ratio** | Return divided by max drawdown — higher means I'm getting paid well for the risk I took |


In [ ]:
# ============================================================
# RISK METRICS — Computing the full suite
# ============================================================
# I'm writing helper functions for the trickier metrics so I can reuse them
# across stocks and portfolios without repeating code.

def compute_sortino(returns, rf_daily):
    """Sortino only penalizes downside moves — I think it's more useful than Sharpe."""
    excess  = returns - rf_daily
    downside = returns[returns < 0]
    down_std = downside.std() * np.sqrt(252) if len(downside) > 0 else 1e-6
    return (excess.mean() * 252 - RISK_FREE_RATE) / down_std

def compute_var_95(returns):
    """Historical VaR — I'm looking at the 5th percentile of daily returns."""
    return -np.percentile(returns, 5)

def compute_max_drawdown(price_series):
    """Max drawdown: the biggest drop from any peak before recovery."""
    rolling_max = price_series.cummax()
    drawdown    = (price_series - rolling_max) / rolling_max
    return drawdown.min()

rf_daily = RISK_FREE_RATE / 252

# ─── Loop through every stock + SPY and compute all metrics ───
# I want one clean table at the end I can look at all at once
risk_metrics = []
for ticker in TICKERS + ['SPY']:
    rets = df_returns[ticker]
    ann_ret = rets.mean() * 252
    ann_vol = rets.std() * np.sqrt(252)
    sharpe  = (ann_ret - RISK_FREE_RATE) / ann_vol
    sortino = compute_sortino(rets, rf_daily)
    var95   = compute_var_95(rets)
    mdd     = compute_max_drawdown(df_prices[ticker])
    calmar  = ann_ret / abs(mdd) if mdd != 0 else 0

    risk_metrics.append({
        'Ticker':    ticker,
        'Ann Return (%)': round(ann_ret * 100, 2),
        'Volatility (%)': round(ann_vol * 100, 2),
        'Sharpe':    round(sharpe, 3),
        'Sortino':   round(sortino, 3),
        'VaR 95% (%)': round(var95 * 100, 3),
        'Max DD (%)': round(mdd * 100, 2),
        'Calmar':    round(calmar, 3)
    })

df_risk = pd.DataFrame(risk_metrics).set_index('Ticker')
print("── Full Risk Metrics Table ──\n")
print(df_risk.to_string())


In [ ]:
# ─── Risk Dashboard — 2x2 grid of key risk visuals ───
# I want to see Sharpe vs Sortino, VaR, max drawdown, and the
# risk-return tradeoff all in one place. Let me put this together.
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

stocks_only = df_risk.drop('SPY')
spy_row     = df_risk.loc['SPY']
colors_bar  = [COLORS['palette'][i] for i in range(len(TICKERS))]

# ── Top-left: Sharpe vs Sortino side by side ──
# I want to compare these two for each stock — Sortino should generally
# be higher since it doesn't penalize upside volatility.
ax = axes[0, 0]
x  = np.arange(len(TICKERS))
w  = 0.35
ax.bar(x - w/2, stocks_only['Sharpe'],  w, color=COLORS['primary'],  edgecolor='white', label='Sharpe')
ax.bar(x + w/2, stocks_only['Sortino'], w, color=COLORS['accent'],   edgecolor='white', label='Sortino')
ax.axhline(spy_row['Sharpe'],  color=COLORS['muted'], ls='--', lw=1.2, label=f'SPY Sharpe ({spy_row["Sharpe"]:.2f})')
ax.set_xticks(x); ax.set_xticklabels(TICKERS)
ax.set_title('Sharpe & Sortino Ratios', fontsize=13, fontweight='bold')
ax.set_ylabel('Ratio')
ax.legend(fontsize=8)

# ── Top-right: VaR at 95% confidence ──
# This tells me the worst daily loss I should expect 5% of the time
ax = axes[0, 1]
var_vals = stocks_only['VaR 95% (%)']
bars = ax.bar(TICKERS, var_vals, color=COLORS['palette'][:len(TICKERS)], edgecolor='white')
ax.axhline(spy_row['VaR 95% (%)'], color=COLORS['muted'], ls='--', lw=1.2,
           label=f'SPY VaR ({spy_row["VaR 95% (%)"]:.2f}%)')
for bar, val in zip(bars, var_vals):
    ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.01,
            f'{val:.2f}%', ha='center', va='bottom', fontsize=8, fontweight='bold')
ax.set_title('Value at Risk (95% Confidence, Daily)', fontsize=13, fontweight='bold')
ax.set_ylabel('VaR (%)')
ax.legend(fontsize=8)

# ── Bottom-left: Max Drawdown ──
# I'm curious which stocks had the scariest drops during the year
ax = axes[1, 0]
mdd_vals = stocks_only['Max DD (%)']
bars = ax.bar(TICKERS, mdd_vals, color=[COLORS['negative']]*len(TICKERS), edgecolor='white')
ax.axhline(spy_row['Max DD (%)'], color=COLORS['muted'], ls='--', lw=1.2,
           label=f'SPY Max DD ({spy_row["Max DD (%)"]:.1f}%)')
for bar, val in zip(bars, mdd_vals):
    ax.text(bar.get_x()+bar.get_width()/2, val - 0.3,
            f'{val:.1f}%', ha='center', va='top', fontsize=8, fontweight='bold', color='white')
ax.set_title('Maximum Drawdown (2025)', fontsize=13, fontweight='bold')
ax.set_ylabel('Drawdown (%)')
ax.legend(fontsize=8)

# ── Bottom-right: Return vs Risk scatter plot ──
# This is my favorite view — it shows the full tradeoff at a glance.
# Stocks in the upper-left corner are the ones I actually want to own.
ax = axes[1, 1]
ax.scatter(stocks_only['Volatility (%)'], stocks_only['Ann Return (%)'],
           c=colors_bar, s=120, edgecolors='white', linewidths=1.5, zorder=5)
for i, t in enumerate(TICKERS):
    ax.annotate(t, (stocks_only.loc[t, 'Volatility (%)'], stocks_only.loc[t, 'Ann Return (%)']),
                xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')
# Plotting SPY as a diamond so it stands out as the benchmark
ax.scatter(spy_row['Volatility (%)'], spy_row['Ann Return (%)'],
           marker='D', s=150, color=COLORS['muted'], edgecolors='white', linewidths=1.5, zorder=5)
ax.annotate('SPY', (spy_row['Volatility (%)'], spy_row['Ann Return (%)']),
            xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold', color=COLORS['muted'])
ax.set_xlabel('Annualized Volatility (%)')
ax.set_ylabel('Annualized Return (%)')
ax.set_title('Risk-Return Tradeoff', fontsize=13, fontweight='bold')

plt.suptitle('Risk Metrics Dashboard — 2025', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('04_risk_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 04_risk_dashboard.png")


---
## 8. Investment Strategy Comparison

Now I want to actually test whether different trading strategies would have performed differently over the year. I'm backtesting five strategies and I'm genuinely curious which one comes out on top:

| Strategy | What I'm testing |
|---|---|
| **Buy & Hold** | The simplest baseline — equal weight everything on day one, never touch it |
| **Monthly Rebalance** | Same equal weight, but I reset back to equal every month |
| **Momentum** | Each month I go long the top-3 stocks by their trailing 30-day return — chasing what's been working |
| **Risk Parity** | I weight each stock inversely by its recent volatility — so the riskier stocks get less allocation |
| **Optimized (Max Sharpe)** | I plug in the weights I solved for in the Markowitz section and hold them |


In [ ]:
# ============================================================
# STRATEGY BACKTESTING — Building the engine
# ============================================================
# I'm writing a generic backtest function that takes any strategy
# and replays it day by day over 2025. Each strategy just needs to
# return a dict of {ticker: weight} for a given date.

def backtest_strategy(strategy_func, name):
    """
    Runs a backtest for any strategy function.
    strategy_func(df_returns, df_prices, date) -> {ticker: weight}
    Returns cumulative portfolio value as a Series.
    """
    portfolio_value = 1.0
    cum_values      = [1.0]
    prev_weights    = strategy_func(df_returns, df_prices, df_returns.index[0])

    for i in range(1, len(df_returns)):
        today_rets = df_returns[TICKERS].iloc[i]
        # The portfolio's daily return is just the weighted sum of stock returns
        port_ret   = sum(prev_weights.get(t, 0) * today_rets[t] for t in TICKERS)
        portfolio_value *= (1 + port_ret)
        cum_values.append(portfolio_value)
        # Ask the strategy what weights to use tomorrow
        prev_weights = strategy_func(df_returns, df_prices, df_returns.index[i])

    return pd.Series(cum_values, index=df_returns.index, name=name)

# ─── Strategy 1: Buy & Hold ───
# The simplest possible baseline — I just equal-weight everything and never touch it
def buy_and_hold(returns, prices, date):
    """Equal weight, set once, never changes."""
    return {t: 1/len(TICKERS) for t in TICKERS}

# ─── Strategy 2: Monthly Rebalance ───
# Same as buy and hold but I reset to equal weight every month.
# I want to see if that periodic rebalancing actually adds value.
def monthly_rebalance(returns, prices, date):
    """Equal weight, rebalanced monthly."""
    return {t: 1/len(TICKERS) for t in TICKERS}

# ─── Strategy 3: Momentum ───
# I'm going to try chasing the winners — each month I look at which
# 3 stocks had the best trailing 30-day return and go all-in on those.
# First 30 days I don't have enough history so I default to equal weight.
_momentum_cache = {}
def momentum_strategy(returns, prices, date):
    """Top-3 stocks by trailing 30-day return, equal weight among them."""
    idx = returns.index.get_loc(date)
    if idx < 30:
        return {t: 1/len(TICKERS) for t in TICKERS}  # warmup period
    trail_30 = returns[TICKERS].iloc[idx-30:idx].sum()
    top3 = trail_30.nlargest(3).index.tolist()
    return {t: (1/3 if t in top3 else 0) for t in TICKERS}

# ─── Strategy 4: Risk Parity ───
# Here I'm flipping the script — instead of chasing returns, I'm
# weighting inversely by volatility. The idea is that riskier stocks
# get less money so each position contributes equally to total risk.
def risk_parity(returns, prices, date):
    """Weight inversely proportional to trailing 60-day volatility."""
    idx = returns.index.get_loc(date)
    if idx < 60:
        return {t: 1/len(TICKERS) for t in TICKERS}  # need 60 days of history first
    trail_60_vol = returns[TICKERS].iloc[idx-60:idx].std()
    inv_vol = 1 / trail_60_vol
    total   = inv_vol.sum()
    return {t: inv_vol[t] / total for t in TICKERS}

print("✓ Strategy functions defined")
print("  Running backtests...")


In [ ]:
# ─── Running all five strategies ───
# Strategy 1: Buy & Hold — the baseline
s1_cum = backtest_strategy(buy_and_hold, 'Buy & Hold')

# Strategy 2: Monthly Rebalance
# I'm handling this one manually month by month because I need to
# actually reset the weights at the start of each month to be accurate.
monthly_cum = [1.0]
monthly_value = 1.0
for month in range(1, 13):
    mask = df_returns.index.month == month
    if mask.sum() == 0: continue
    month_rets = df_returns[TICKERS].loc[mask]
    w_eq = np.array([1/len(TICKERS)] * len(TICKERS))  # reset to equal weight
    port_month_rets = month_rets.values @ w_eq
    for dr in port_month_rets:
        monthly_value *= (1 + dr)
        monthly_cum.append(monthly_value)
s2_cum = pd.Series(monthly_cum[:len(df_returns)], index=df_returns.index, name='Monthly Rebalance')

# Strategy 3: Momentum — chasing the trailing winners
s3_cum = backtest_strategy(momentum_strategy, 'Momentum (Top-3)')

# Strategy 4: Risk Parity — risk-weighted allocation
s4_cum = backtest_strategy(risk_parity, 'Risk Parity')

# Strategy 5: Max Sharpe — using the optimized weights I solved for earlier.
# I'm holding those weights for the full year to see how they actually performed.
def optimized_maxsharpe(returns, prices, date):
    return {t: max_sharpe_weights[i] for i, t in enumerate(TICKERS)}
s5_cum = backtest_strategy(optimized_maxsharpe, 'Max Sharpe (Optimized)')

# SPY benchmark for comparison
spy_rets = df_returns['SPY']
spy_cum  = (1 + spy_rets).cumprod()
spy_cum  = spy_cum / spy_cum.iloc[0]
spy_cum.name = 'SPY Benchmark'

# Putting everything into one DataFrame for easy comparison
strategies = pd.DataFrame({
    'Buy & Hold':          s1_cum,
    'Monthly Rebalance':   s2_cum,
    'Momentum (Top-3)':    s3_cum,
    'Risk Parity':         s4_cum,
    'Max Sharpe':          s5_cum,
    'SPY Benchmark':       spy_cum
})

print("✓ All 5 strategies backtested")
print(f"\n  Final values (per $1 invested Jan 1):\n")
print(strategies.iloc[-1].sort_values(ascending=False).to_string())


In [ ]:
# ─── Computing risk-adjusted metrics for each strategy ───
# I want to compare them on more than just total return — I need to see
# the Sharpe, drawdown, and Calmar to really understand the tradeoffs.
strat_metrics = []
for col in strategies.columns:
    daily_ret = strategies[col].pct_change().dropna()
    ann_ret   = daily_ret.mean() * 252
    ann_vol   = daily_ret.std() * np.sqrt(252)
    sharpe    = (ann_ret - RISK_FREE_RATE) / ann_vol if ann_vol > 0 else 0
    mdd       = compute_max_drawdown(strategies[col])
    calmar    = ann_ret / abs(mdd) if mdd != 0 else 0
    total_ret = (strategies[col].iloc[-1] - 1) * 100

    strat_metrics.append({
        'Strategy':      col,
        'Total Return (%)': round(total_ret, 2),
        'Ann Return (%)':   round(ann_ret * 100, 2),
        'Volatility (%)':   round(ann_vol * 100, 2),
        'Sharpe Ratio':     round(sharpe, 3),
        'Max Drawdown (%)': round(mdd * 100, 2),
        'Calmar Ratio':     round(calmar, 3)
    })

df_strat = pd.DataFrame(strat_metrics).set_index('Strategy').sort_values('Sharpe Ratio', ascending=False)
print("── Strategy Comparison Metrics ──\n")
print(df_strat.to_string())


In [ ]:
# ─── Strategy Comparison Dashboard ───
# I want three views: cumulative return lines, total return bars,
# and a Sharpe vs Drawdown scatter to see the risk-reward picture.
fig = plt.figure(figsize=(16, 12))
gs  = gridspec.GridSpec(2, 2, hspace=0.3, wspace=0.25)

# Color map so each strategy has a consistent color across all plots
strat_colors = {
    'Buy & Hold':        COLORS['palette'][0],
    'Monthly Rebalance': COLORS['palette'][1],
    'Momentum (Top-3)':  COLORS['palette'][2],
    'Risk Parity':       COLORS['palette'][3],
    'Max Sharpe':        COLORS['positive'],
    'SPY Benchmark':     COLORS['muted']
}

# ── Top panel: cumulative returns over the full year ──
# I'm making Max Sharpe and SPY thicker so they stand out as the key comparison
ax1 = fig.add_subplot(gs[0, :])
for col in strategies.columns:
    lw = 2.8 if col in ('Max Sharpe', 'SPY Benchmark') else 1.6
    ls = '--' if col == 'SPY Benchmark' else '-'
    ax1.plot(strategies.index, (strategies[col] - 1) * 100,
             color=strat_colors[col], linewidth=lw, linestyle=ls, label=col)

ax1.axhline(0, color='gray', linewidth=0.8, linestyle=':')
ax1.set_title('Cumulative Returns by Strategy (2025)', fontsize=15, fontweight='bold')
ax1.set_ylabel('Return (%)')
ax1.legend(loc='upper left', fontsize=9, framealpha=0.95)

# ── Bottom-left: total return by strategy ──
ax2 = fig.add_subplot(gs[1, 0])
strat_order  = df_strat.sort_values('Total Return (%)', ascending=True).index
x_pos        = np.arange(len(strat_order))
bar_colors   = [strat_colors[s] for s in strat_order]

bars = ax2.barh(x_pos, df_strat.loc[strat_order, 'Total Return (%)'], color=bar_colors, edgecolor='white', height=0.5)
for bar, val in zip(bars, df_strat.loc[strat_order, 'Total Return (%)']):
    ax2.text(val + 0.3, bar.get_y() + bar.get_height()/2,
             f'{val:.1f}%', va='center', fontsize=9, fontweight='bold')
ax2.set_yticks(x_pos)
ax2.set_yticklabels(strat_order, fontsize=10)
ax2.set_title('Total Return by Strategy', fontsize=13, fontweight='bold')
ax2.set_xlabel('Total Return (%)')

# ── Bottom-right: Sharpe vs Max Drawdown ──
# This is the key "did it earn enough to justify the pain?" view
ax3 = fig.add_subplot(gs[1, 1])
for s in df_strat.index:
    marker = 'D' if s == 'SPY Benchmark' else 'o'
    ax3.scatter(abs(df_strat.loc[s, 'Max Drawdown (%)']),
                df_strat.loc[s, 'Sharpe Ratio'],
                color=strat_colors[s], s=140, marker=marker,
                edgecolors='white', linewidths=1.5, zorder=5)
    ax3.annotate(s, (abs(df_strat.loc[s, 'Max Drawdown (%)']), df_strat.loc[s, 'Sharpe Ratio']),
                 xytext=(6, 4), textcoords='offset points', fontsize=8.5, fontweight='bold')

ax3.set_xlabel('|Max Drawdown| (%)', fontsize=11)
ax3.set_ylabel('Sharpe Ratio', fontsize=11)
ax3.set_title('Risk-Reward: Sharpe vs Drawdown', fontsize=13, fontweight='bold')

plt.savefig('05_strategy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 05_strategy_comparison.png")


---
## 9. Executive Summary

I want to wrap everything up in one dashboard that tells the story at a glance — best performers, the optimal portfolio allocation, which strategy actually won, and a drawdown timeline so I can see when things got scary.


In [ ]:
# ============================================================
# EXECUTIVE SUMMARY — Wrapping everything up in one dashboard
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# ── Top-left: Top 5 stocks by return ──
# I want to see at a glance who the winners were, with SPY as a reference line
ax = axes[0, 0]
top5 = full_year_return[TICKERS].nlargest(5).sort_values(ascending=True)
bars = ax.barh(top5.index, top5.values,
               color=[COLORS['positive'] if v > 0 else COLORS['negative'] for v in top5.values],
               edgecolor='white', height=0.5)
ax.axvline(full_year_return['SPY'], color=COLORS['muted'], ls='--', lw=1.5,
           label=f'SPY ({full_year_return["SPY"]:.1f}%)')
for bar, val in zip(bars, top5.values):
    ax.text(val + 0.2, bar.get_y() + bar.get_height()/2,
            f'{val:.1f}%', va='center', fontsize=9, fontweight='bold')
ax.set_title('Top 5 Stocks by 2025 Return', fontsize=13, fontweight='bold')
ax.legend(fontsize=9)

# ── Top-right: Max Sharpe portfolio allocation as a pie chart ──
# I'm curious to see the final allocation visually — which stocks
# did the optimizer actually want to own?
ax = axes[0, 1]
wedge_colors = COLORS['palette'][:len(TICKERS)]
explode = [0.04] * len(TICKERS)
wedges, texts, autotexts = ax.pie(
    max_sharpe_weights, labels=TICKERS, autopct='%1.1f%%',
    colors=wedge_colors, explode=explode, startangle=90,
    textprops={'fontsize': 9}
)
for at in autotexts:
    at.set_fontsize(8)
    at.set_fontweight('bold')
ax.set_title('Max Sharpe Portfolio — Allocation', fontsize=13, fontweight='bold')

# ── Bottom-left: Key findings text box ──
# Pulling the winners from each category so I can summarize the takeaways
ax = axes[1, 0]
ax.axis('off')
best_return  = df_strat['Total Return (%)'].idxmax()
best_sharpe  = df_strat['Sharpe Ratio'].idxmax()
best_calmar  = df_strat['Calmar Ratio'].idxmax()
least_mdd    = df_strat['Max Drawdown (%)'].idxmax()

summary_text = (
    f"KEY FINDINGS\n"
    f"{'─'*38}\n\n"
    f"🏆  Best Total Return:     {best_return}\n"
    f"        ({df_strat.loc[best_return, 'Total Return (%)']:.1f}%)\n\n"
    f"📊  Best Risk-Adjusted:    {best_sharpe}\n"
    f"        (Sharpe: {df_strat.loc[best_sharpe, 'Sharpe Ratio']:.3f})\n\n"
    f"🛡️   Safest (Calmar):      {best_calmar}\n"
    f"        (Calmar: {df_strat.loc[best_calmar, 'Calmar Ratio']:.3f})\n\n"
    f"📈  Max Sharpe Portfolio\n"
    f"        Return: {portfolio_return(max_sharpe_weights)*100:.1f}%  |  "
    f"Vol: {portfolio_volatility(max_sharpe_weights)*100:.1f}%  |  "
    f"Sharpe: {sharpe_ms:.3f}"
)
ax.text(0.05, 0.95, summary_text, transform=ax.transAxes,
        fontsize=11, verticalalignment='top', fontfamily='monospace',
        bbox=dict(boxstyle='round,pad=0.5', facecolor=COLORS['palette'][0], alpha=0.08, edgecolor=COLORS['primary']))

# ── Bottom-right: Drawdown timeline for the best risk-adjusted strategy ──
# I want to see when that strategy actually got hit hardest during the year
ax = axes[1, 1]
best_strat_col = df_strat['Sharpe Ratio'].idxmax()
best_series    = strategies[best_strat_col]
rolling_max    = best_series.cummax()
drawdown       = (best_series - rolling_max) / rolling_max * 100

ax.fill_between(drawdown.index, drawdown.values, 0, color=COLORS['negative'], alpha=0.35)
ax.plot(drawdown.index, drawdown.values, color=COLORS['negative'], linewidth=1.2)
ax.set_title(f'Drawdown Timeline — {best_strat_col}', fontsize=13, fontweight='bold')
ax.set_ylabel('Drawdown (%)')

plt.suptitle('2025 Investment Analysis — Executive Summary',
             fontsize=17, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('06_executive_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Saved: 06_executive_summary.png")
print("\n" + "="*60)
print("  ANALYSIS COMPLETE")
print("="*60)
